In [11]:
from google.colab import files
uploaded = files.upload()

Saving nba_master_all_seasons_sorted.csv to nba_master_all_seasons_sorted (1).csv


# 01 — Data Audit: NBA Three-Point Inflation

## Kontext
Dieses Notebook prüft die Datenqualität der für das Projekt verwendeten Datenbasis.
Die Rohdaten liegen auf **Team × Spiel**-Ebene (Team-Game-Logs) vor und werden in späteren Notebooks
auf **Team × Saison** aggregiert. Details zu Quellen und Datenpipeline stehen im README.

## Ziele
- Überblick über Struktur und Umfang des Datensatzes
- Prüfung auf fehlende Werte und Duplikate
- Plausibilitätschecks zentraler Variablen (3PA, 3PM, 3P%)
- Konvertierung von Prozentspalten in numerische Werte

In [12]:
import pandas as pd
import numpy as np

df = pd.read_csv(
    "/content/nba_master_all_seasons_sorted.csv",
    sep=";",
    decimal=",",
    engine="python"
)

df.head()

,gameid,date,type,teamid,team,home,away,MIN,PTS,FGM,...,DREB,REB,AST,TOV,STL,BLK,PF,+/-,win,season
0,29600001,01.11.96,regular,1610612738,BOS,BOS,CHI,48,98,38,...,22,36,20,18.0,10,2,33,-9.0,0,1997
1,29600001,01.11.96,regular,1610612741,CHI,BOS,CHI,48,107,42,...,29,37,28,19.0,7,8,23,9.0,1,1997
2,29600002,01.11.96,regular,1610612739,CLE,NJN,CLE,48,90,34,...,23,35,16,15.0,11,1,24,13.0,1,1997
3,29600002,01.11.96,regular,1610612751,NJN,NJN,CLE,48,77,23,...,24,35,13,22.0,7,7,19,-13.0,0,1997
4,29600003,01.11.96,regular,1610612749,MIL,PHI,MIL,48,111,38,...,31,50,21,15.0,9,7,30,8.0,1,1997


In [13]:
df.shape
list(df.columns)

['gameid',
 'date',
 'type',
 'teamid',
 'team',
 'home',
 'away',
 'MIN',
 'PTS',
 'FGM',
 'FGA',
 'FG%',
 '3PM',
 '3PA',
 '3P%',
 'FTM',
 'FTA',
 'FT%',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'TOV',
 'STL',
 'BLK',
 'PF',
 '+/-',
 'win',
 'season']

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71970 entries, 0 to 71969
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   gameid  71970 non-null  int64  
 1   date    71970 non-null  object 
 2   type    71970 non-null  object 
 3   teamid  71970 non-null  int64  
 4   team    71970 non-null  object 
 5   home    71970 non-null  object 
 6   away    71970 non-null  object 
 7   MIN     71970 non-null  int64  
 8   PTS     71970 non-null  int64  
 9   FGM     71970 non-null  int64  
 10  FGA     71970 non-null  int64  
 11  FG%     71970 non-null  object 
 12  3PM     71970 non-null  int64  
 13  3PA     71970 non-null  int64  
 14  3P%     71970 non-null  object 
 15  FTM     71970 non-null  int64  
 16  FTA     71970 non-null  int64  
 17  FT%     71970 non-null  object 
 18  OREB    71970 non-null  int64  
 19  DREB    71970 non-null  int64  
 20  REB     71970 non-null  int64  
 21  AST     71970 non-null  int64  
 22

In [15]:
missing = df.isna().sum().sort_values(ascending=False)
missing[missing > 0]

,0


In [16]:
# Eindeutigkeit: pro Spiel (gameid) gibt es je Team genau eine Zeile
df.duplicated(subset=["gameid", "teamid"]).sum()
# Plausibilitätscheck: pro gameid sollten typischerweise 2 Teams vorkommen
df.groupby("gameid")["teamid"].nunique().value_counts().head()

,count
teamid,
2,35985


In [17]:
(df["3PM"] > df["3PA"]).sum()

np.int64(0)

In [18]:
pct_cols = ["FG%", "3P%", "FT%"]

# Robust: falls sie als strings mit Komma vorliegen
for c in pct_cols:
    df[c] = pd.to_numeric(df[c].astype(str).str.replace(",", ".", regex=False), errors="coerce")

df[pct_cols].dtypes

,0
FG%,float64
3P%,float64
FT%,float64


In [19]:
df[pct_cols].describe().loc[["min", "max"]]
df["MIN"].describe()

,MIN
count,71970.000000
mean,48.339933
std,1.464890
min,47.000000
25%,48.000000
50%,48.000000
75%,48.000000
max,68.000000


## Audit-Fazit

- Der Datensatz umfasst **71970** Beobachtungen und **29** Variablen (Team × Spiel).
- Es wurden **keine bzw. nur wenige fehlende Werte** gefunden (siehe Missing-Check).
- Die Kombination **(gameid, teamid)** ist eindeutig (keine Duplikate).
- Zentrale Plausibilitätschecks sind erfüllt (u. a. 3PM ≤ 3PA).
- Prozentwerte (FG%, 3P%, FT%) wurden erfolgreich in numerische Werte konvertiert.

Der Datensatz ist damit als Grundlage für die Aggregation auf Team×Saison sowie für Trendanalyse und Hypothesentests geeignet.